In [ ]:
import os, sys
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [ ]:
conn = psycopg2.connect(dbname = "operations_manager_prod", user = "sarthak_sachdev", password = "Sarthak@12345", host = "operation.replica.upgrid.in", port = "5432")

print('PostgreSQL Connection Established')

query1 = '''
select id from logs where to_value = 'completed' and (created_at + INTERVAL '330 minutes')::DATE >= '20250401'
'''

dfTicketLogs = pd.read_sql(query1, conn)

query2 = '''
select id, parent_id as task_id from tasks where (created_at + INTERVAL '330 minutes')::DATE >= '20250401'
'''

dfTasks = pd.read_sql(query2, conn)

query3 = '''
select task_id, slug as ticket_id from tickets where (created_at + INTERVAL '330 minutes')::DATE >= '20250401'
'''

dfTicket = pd.read_sql(query3, conn)

conn.close()

dfTicketLogs

In [ ]:
dfTasks

In [ ]:
dfTicket

In [ ]:
merged = dfTicketLogs.merge(dfTasks, how='left', on='id').merge(dfTicket, how='left', on='task_id')

merged = merged.dropna()

merged

In [ ]:
dfTickets = adbFetch("""select ticket_id, complainant_zone as zone from ticketModelNew where assigned_to_team = 'F-Ops' and date >= '20250401' and source in ('Inbound', 'Whatsapp', 'driverApp') and complainant_type = 'driver' and complainant_zone in ('MVA03', 'MVA05')""")

dfTickets

In [ ]:
ticketsMerged = dfTickets.merge(merged, how = 'left', on='ticket_id')

ticketsMerged = ticketsMerged.dropna()

ticketsMerged

In [ ]:
ticketsGrouped = ticketsMerged.groupby('zone').agg(
    zonewiseTicketCount = ('ticket_id', 'count') 
)

ticketsGrouped